In [1]:
# Load the data
import os
import pandas as pd
import pickle

xu_masses = [500, 750, 1000, 1500, 2000, 2500]  # GeV
phi_masses = [1, 5, 10, 50, 100, 325]  # GeV
mc_names = ["signal", "ttbarmumu", "ttbarmumumunu"]

data_path = os.path.join(os.getcwd(), "data")
print("Loading data from", data_path)

data = {
    "xu_{}_phi_{}".format(xu_mass, phi_mass): {
        f"{mc}": pd.read_csv(
            os.path.join(
                data_path,
                f"xu_{xu_mass}_GeV",
                f"phi_{phi_mass}_GeV",
                f"{mc}_hist.dat",
            ),
            sep=" ",
            names=["bin_center", "bin_content"],
            header=None,
        )
        for mc in mc_names
    }
    for xu_mass in xu_masses
    for phi_mass in phi_masses
}
print("Done!")

Loading data from /home/c.rodriguez45/github/U1T3R_Significances/data
Done!


In [2]:
from itertools import product

# This transformation was necessary due to a data storage error.
# Originally, the data was stored in a way that the 'bin_center' values were in
#  reverse order and the 'bin_content' values were stored as their complements
# (i.e., 1 - original value), and in the wrong bin order.
for key, mc in product(data.keys(), mc_names):
    if mc == "signal":
        continue
    if "2500" in key :
        transformed_data = [
            [
                data[key][mc]["bin_center"][49 - x],
                1 - data[key][mc]["bin_content"][49 - x],
            ]
            for x in range(50)
        ]
    else : 
        transformed_data = [
            [
                data[key][mc]["bin_center"][49 - x],
                1 - data[key][mc]["bin_content"][x],
            ]
            for x in range(50)
        ]
    data[key][mc] = pd.DataFrame(
        transformed_data, columns=["bin_center", "bin_content"]
    )

In [3]:
data["xu_2500_phi_325"]["ttbarmumu"].head()


,bin_center,bin_content
0,0.01,0.000006
1,0.03,0.000007
2,0.05,0.000007
3,0.07,0.000007
4,0.09,0.000008


In [4]:
merged_data = {}
for key in data:
    merged_df = pd.merge(
        data[key]["signal"], data[key]["ttbarmumu"], on="bin_center"
    )
    merged_df = pd.merge(
        merged_df, data[key]["ttbarmumumunu"], on="bin_center"
    )
    merged_df.columns = ["bin_center", "signal", "ttbarmumu", "ttbarmumumunu"]
    merged_data[key] = merged_df



In [5]:
print(merged_data["xu_500_phi_1"].head())
print(merged_data["xu_500_phi_1"].tail())

   bin_center    signal  ttbarmumu  ttbarmumumunu
0        0.01  0.996261   0.000021   1.151861e-07
1        0.03  0.001830   0.000009   1.175761e-07
2        0.05  0.000550   0.000010   1.200624e-07
3        0.07  0.000270   0.000010   1.226509e-07
4        0.09  0.000250   0.000010   1.253482e-07
    bin_center    signal  ttbarmumu  ttbarmumumunu
45        0.91  0.000002   0.000044       0.000001
46        0.93  0.000002   0.000040       0.000001
47        0.95  0.000002   0.000170       0.000002
48        0.97  0.000002   0.000440       0.000010
49        0.99  0.000002   0.998468       0.999973


In [6]:
from io import StringIO
import pandas as pd
data = """
0.1292000000	0.0613100000	0.0299900000	0.0073040000	0.0016790000	0.0004024
0.1291000000	0.0612500000	0.0299600000	0.0072960000	0.0016760000	0.0004024
0.1291000000	0.0612400000	0.0299500000	0.0072960000	0.0016740000	0.0004132
0.1262000000	0.0606400000	0.0298000000	0.0072800000	0.0016710000	0.0004012
0.1172000000	0.0589500000	0.0293400000	0.0072260000	0.0016650000	0.0004000
0.0089690000	0.0388800000	0.0237400000	0.0066090000	0.0015850000	0.0003100
"""

# Read the data into a DataFrame
xs_df = pd.read_csv(StringIO(data), sep="\t", header=None)

# Set the column names to the xu masses
xs_df.columns = [f"xu_{int(x)}_GeV" for x in xu_masses]

# Set the index to the phi masses
xs_df.index = [f"phi_{x}_GeV" for x in phi_masses]

# Save the DataFrame to a pickle file
with open(os.path.join(data_path, "cross_sections.pkl"), "wb") as f:
    pickle.dump(xs_df, f)

In [7]:
lumino = 150 / 1e-3  # 1/fb to 1/pb
eff = 1.0  # Assume 100% efficiency for now
mc_names = ["signal", "ttbarmumu", "ttbarmumumunu"]
for xu_mass, phi_mass in product(xu_masses, phi_masses):
    xs_dict = {}
    xs_dict["signal"] = xs_df.loc[
        f"phi_{phi_mass}_GeV", f"xu_{xu_mass}_GeV"
    ]  # pb
    xs_dict["ttbarmumu"] = 0.002574  # pb
    xs_dict["ttbarmumumunu"] = 0.0004692  # pb

    for mc in mc_names:
        merged_data[f"xu_{xu_mass}_phi_{phi_mass}"][mc] *= (
            xs_dict[mc] * lumino * eff
        )
    merged_data[f"xu_{xu_mass}_phi_{phi_mass}"]["background"] = (
        merged_data[f"xu_{xu_mass}_phi_{phi_mass}"]["ttbarmumu"]
        + merged_data[f"xu_{xu_mass}_phi_{phi_mass}"]["ttbarmumumunu"]
    )

# Save the data
with open(os.path.join(data_path, "histograms.pkl"), "wb") as f:
    pickle.dump(merged_data, f)

In [8]:
print(merged_data["xu_2500_phi_1"].head())
print(merged_data["xu_2500_phi_1"].tail())

   bin_center     signal  ttbarmumu  ttbarmumumunu  background
0        0.01  58.724246   0.003314       0.000373    0.003686
1        0.03   0.722413   0.004702       0.000388    0.005091
2        0.05   0.277619   0.003599       0.000405    0.004003
3        0.07   0.151483   0.003755       0.000422    0.004177
4        0.09   0.094149   0.003922       0.000441    0.004363
    bin_center    signal   ttbarmumu  ttbarmumumunu  background
45        0.91  0.001173    0.335886       0.035188    0.371074
46        0.93  0.001123    0.544368       0.049263    0.593630
47        0.95  0.001076    0.803039       0.104155    0.907194
48        0.97  0.001032    2.250825       0.242794    2.493619
49        0.99  0.000991  380.817975      69.798786  450.616761
